# Biogas Production Calculations

## Introduction
One major section of the UASB design is production of biogas, a mixture of methane, carbon dioxide, and other trace gases produced from anaerobic bacteria within the bed.  This biogas can be burned, similar to natural gas and used for heating, evergy production, or cooking.  One goal of the UASB design is to maximize biogas production, and provide an easy system to store, access, and use the biogas in small scale communities. These calculations serve to characterize the production of this biogas, and calculate the necessary parameters for our gas capture lid.  

## Biogas Capture Design

Add images and description of biogas capture system.  Describe design process and key ideas behind it

To properly capture gas rising through the UASB reactor, we utilize a gas capture headspace built into the top of the reactor.

## Design Parameters

Here we set a series of parameters for the design of our pilot scale reactor.  These values are subject to change, but they offer our current ideas on the size and function of our reactor.  The parameters and the reasons for their decision is detailed below.

1. Height Reactor: 7 ft , max allowed to fabricate in lab safely
2. Diameter Reactor: 3 ft, based on 1 L/s plant design
3. Height sludge blanket: 3.5 ft, assumed half height of reactor
4. Height Headspace: TBD from production calculations
5. Diameter Headspace: TBD from production calculations
6. COD_Load: 750 mg/L, medium value from most plants (can be varied)

## Critical Assumptions

In designing the biogas capture, there are many areas of uncertainty that are challenging to characterize before we can collect data from our own reactor.  In order to minimize uncertainty and provide the most accurate calculations, we used a series of assumptions for these areas.  The assumed values, and their justification and sources are detailed below.

1. WW_gen: 3 ml/s, rule of thumb for wastewater production per person, from Monroe
2. HRT: 4 hours, Number taken from previous lab scale tests and literature reviews
3. COD_rem: 70%, Assumed efficency of reactor for breaking down COD, general number from the literature
4. BG_Max: 37.8% of COD_rem, calculated from theoretical efficency, using stoichiometry of reaction.  Detailed in Spring 2014 UASB team report
5. BG_Min: 6%, Minimum biogas production from COD load, taken from lower bound in [Van Lier report](https://courses.edx.org/c4x/DelftX/CTB3365STx/asset/Chap_4_Van_Lier_et_al.pdf)
6. BG_Avg: 16%, Middle value, taken from average output in Aguaclara reactors (in Spring 2014 UASB team report)
7. SH_high: Assuming maximum efficency of a stove, using results from [SSWM](https://www.sswm.info/content/direct-use-biogas)
8. SH_low: Assuming minimum efficency of a stove, using results from [SSWM](https://www.sswm.info/content/direct-use-biogas)

## Current to-do

1. Add new volume adjustments for sloped bottom
2. Run calculation with set initial parameters
3. Find height (and diameter) value that provides desired offgassing rate (every 1-2 days) and does not constrain tube settler or sludge weir (less than 2.5 feet)
4. Use these values to determine tube settler and sludge weir location/geometry

5. Edit the returns on the functions to return arrays

# Size Calculations (Will become seperate python file)

In [19]:
from aide_design.play import*
import math

def UASBSize(diam, height):
    """Returns the flow rate of the UASB reactor given a cylindical body. 
    Assumes that half the reactor contains the settled bed, which is used for the HRT. 
    Takes the inputs of diameter and height."""
    
    WW_gen = 3 * u.mL/u.s        #Wastewater generated per person, rule of thumb from Monroe
    WW_gen_bw = 0.6 * u.mL/u.s   #Assumes 20% of mixed wastewater
    HRT = 4 * u.hr               #Hydraulic Residence Time, determined from lab scale tests
    
    angle = 60 * u.deg 
    height_cyl_hoof = diam/2 * np.tan(angle) 
    vol_cyl_hoof = 2/3 * (diam/2)**2 * height_cyl_hoof
    vol_reactor = (math.pi * (diam / 2)**2 * height / 2) - (2 * vol_cyl_hoof)
    
    flow = vol_reactor / HRT
    people_served = int(flow / WW_gen)       #People served per reactor
    people_served_BW = int(flow / WW_gen_bw) #People served per reactor treating only blackwater
    
    print('The flow rate of the reactor is', flow.to(u.L/u.s))
    print('The number of people served by this reactor is', people_served)
    print('The number of people served by this reactor if only blackwater is treated is', people_served_BW)
    return [flow, vol_reactor]

## Check on Size Calculation

In [20]:
UASBSize(3 * u.ft, 7 * u.ft)

The flow rate of the reactor is 0.03332 liter / second
The number of people served by this reactor is 11
The number of people served by this reactor if only blackwater is treated is 55


[<Quantity(4.236453378239919, 'foot ** 3 / hour')>,
 <Quantity(16.945813512959678, 'foot ** 3')>]

# Biogas Calculations

In [21]:
#Calculate Biogas Rate of Production (L/s) in UASB
def BiogasProd(Flow, COD_Load):
    COD = Flow * COD_Load
    COD_rem = COD * 0.7 #Assuming x% efficency of COD removal and conversion in reactor
    BGMax = COD_rem * 0.378 *(u.ml/u.mg) #Theoretical Productiom, from Fall 2014 UASB team report
    BGMin = COD_rem * 0.06 *(u.ml/u.mg) #Production using minimum value from Van Lier report
    BGAvg = COD_rem * 0.16 *(u.ml/u.mg) #Production using average value from Spring 2014 tests
    return [BGMax, BGAvg, BGMin]

#Calculate Energy Production from Biogas Produced
def EnergyProduction(Biogas):
    Biogas = Biogas.to(u.L/u.days)
    SH_high = Biogas / (200 * u.L) #Hours of stove usage, given maximum efficency of stove
    SH_low = Biogas / (450 * u.L) #Hours of stove usage, given minimum efficency
    #KWH = Biogas / (700 * u.L/u.kwh) #Kilowatt Hours generated from biogas used 
    return [SH_high,SH_low]

## Check on Biogas Produced

In [22]:
Biogas = BiogasProd(0.01 * u.L/u.s, 7300 * u.mg/u.L)
BGMax = Biogas[0]
BGAvg = Biogas[1]
BGMin = Biogas[2]
print("Maximum biogas production is", BGMax, "\n" "Average biogas production is", BGAvg, "\n" 
      "Minimum biogas production is", BGMin)

Maximum biogas production is 19.32 milliliter / second 
Average biogas production is 8.176 milliliter / second 
Minimum biogas production is 3.066 milliliter / second


## Check on Energy Production

In [24]:
NRGMax = EnergyProduction(BGMax)
NRGAvg = EnergyProduction(BGAvg)
NRGMin = EnergyProduction(BGMin)

print("Maximum biogas production is between", NRGMax[0], "and", NRGMax[1], "\n" 
      "Average biogas production is between", NRGAvg[0], "and", NRGAvg[1], "\n" 
      "Minimum biogas production is between", NRGMin[0]


Maximum biogas production is between 8.344 / day and 3.709 / day 
Average biogas production is between [<Quantity(3.5320319999999996, '1 / day')>, <Quantity(1.5697919999999999, '1 / day')>] 
Minimum biogas production is between [<Quantity(1.3245119999999997, '1 / day')>, <Quantity(0.5886719999999999, '1 / day')>]


# Headspace Design

To properly capture and store biogas produced before usage in homes, a headspace will be utilized in the UASB.  This top layer will collect gas rising through the UASB, and store it within until a manual offgassing event moves the gas to further storage or use.

In [10]:
#Filltime: determine time taken to fill headspace and push water completely downwards below level of seal
#time_offgas = Maximum time allows before offgasing event that returns water level to original height
#Offgas_Vol: Volume of gas released in each offgassing event 
def filltime(Headspace_Diam, Headspace_Height, WaterLevel, BiogasRate):
    Vol0 = (math.pi * (Headspace_Diam / 2)**2 * (Headspace_Height - WaterLevel)) #volume of headspace without water initially
    VolW = (math.pi * (Headspace_Diam / 2)**2 * WaterLevel) #Volume of water initially within headspace
    time_offgas = VolW / BiogasRate
    Offgas_Vol = Vol0 + VolW
    return [time_offgas, Offgas_Vol]

## Check for filltime

In [14]:
Offgas = filltime(2.5 * u.ft, 3 * u.ft, 3 * u.ft, 1 * u.ft**3/u.day)
Time_Offgas = Offgas[0]
Vol_Offgas = Offgas[1]
print("The time between offgas event is", Time_Offgas, "\n", "The volume builtup between offgas events is", Vol_Offgas)

NameError: name 'u' is not defined

## Calculations using Design Assumptions

In [12]:
#Size and Biogas Production
(Volume, Flowrate) = ReactorSize(3 * u.ft, 3.5 * u.ft)
(Max, Min, Biogas_Avg) = BiogasProd(Flowrate, 750 * (u.mg / u.L))
print(Volume)
print(Flowrate)
print(Avg)

NameError: name 'ReactorSize' is not defined

In [13]:
#Headspace
(time, vol) = filltime(2 * u.ft, 1 * u.ft, 0.5 * u.ft, Biogas_Avg.to(u.ft**3 / u.day))
print(time, vol)

NameError: name 'u' is not defined